# ETL DataSUS

ELT com PySUS

Author: Rodrigo Barreiro

# Testing if PySUS is working

In [1]:
import pandas as pd
from collections import Counter
from pathlib import Path
import re

In [2]:
from pysus import SIA
sia = SIA().load() # Loads the files from DATASUS

In [3]:
sia

SIA - Sistema de Informações Ambulatoriais

In [4]:
sia.metadata

{'long_name': 'Sistema de Informações Ambulatoriais',
 'source': 'http://sia.datasus.gov.br/principal/index.php',
 'description': 'O Sistema de Informação Ambulatorial (SIA) foi instituído pela Portaria GM/MS n.º 896 de 29 de junho de 1990. Originalmente, o SIA foi concebido a partir do projeto SICAPS (Sistema de Informação e Controle Ambulatorial da Previdência Social), em que os conceitos, os objetivos e as diretrizes criados para o desenvolvimento do SICAPS foram extremamente importantes e amplamente utilizados para o desenvolvimento do SIA, tais como: (i) o acompanhamento das programações físicas e orçamentárias; (ii) o acompanhamento das ações de saúde produzidas; (iii) a agilização do pagamento e controle orçamentário e financeiro; e (iv) a formação de banco de dados para contribuir com a construção do SUS.'}

In [5]:
sia.groups

{'AB': 'APAC de Cirurgia Bariátrica',
 'ABO': 'APAC de Acompanhamento Pós Cirurgia Bariátrica',
 'ACF': 'APAC de Confecção de Fístula',
 'AD': 'APAC de Laudos Diversos',
 'AM': 'APAC de Medicamentos',
 'AMP': 'APAC de Acompanhamento Multiprofissional',
 'AN': 'APAC de Nefrologia',
 'AQ': 'APAC de Quimioterapia',
 'AR': 'APAC de Radioterapia',
 'ATD': 'APAC de Tratamento Dialítico',
 'BI': 'Boletim de Produção Ambulatorial individualizado',
 'IMPBO': '',
 'PA': 'Produção Ambulatorial',
 'PAM': '',
 'PAR': '',
 'PAS': '',
 'PS': 'RAAS Psicossocial',
 'SAD': 'RAAS de Atenção Domiciliar'}

In [6]:
print(f'All files for APAC quimioterapia: {len(sia.get_files(["AQ"]))}')

All files for APAC quimioterapia: 5521


## Calculating Files and Size

In [7]:
sia.get_files("AQ", uf="SP", year=2022)

[AQSP2201.dbc,
 AQSP2202.dbc,
 AQSP2203.dbc,
 AQSP2204.dbc,
 AQSP2205.dbc,
 AQSP2206.dbc,
 AQSP2207.dbc,
 AQSP2208.dbc,
 AQSP2209.dbc,
 AQSP2210.dbc,
 AQSP2211.dbc,
 AQSP2212.dbc]

In [8]:
all_apac_quimio_sia = len(sia.get_files("AQ", year=2022))
print(f'Number of files for AQ in 2022: {all_apac_quimio_sia}')

Number of files for AQ in 2022: 321


In [9]:
aq_files_2022 = sia.get_files("AQ", year=2022)
Counter([ str(x)[2:4] for x in aq_files_2022 ])

Counter({'AC': 12,
         'AL': 12,
         'AM': 12,
         'BA': 12,
         'CE': 12,
         'DF': 12,
         'ES': 12,
         'GO': 12,
         'MA': 12,
         'MG': 12,
         'MS': 12,
         'MT': 12,
         'PA': 12,
         'PB': 12,
         'PE': 12,
         'PI': 12,
         'PR': 12,
         'RJ': 12,
         'RN': 12,
         'RO': 12,
         'RS': 12,
         'SC': 12,
         'SE': 12,
         'SP': 12,
         'TO': 12,
         'AP': 11,
         'RR': 10})

In [10]:
# Countig file size
files = sia.get_files("AQ", year=2022)
sizes = [ sia.describe(my_file)['size'] for my_file in files ]


# Example list
sizes = ['37.9 kB', '1.2 MB', '550 B', '2.1 MB', '900 kB']

# Conversion to bytes
unit_multipliers = {
    'B': 1,
    'kB': 1024,
    'MB': 1024 ** 2,
    'GB': 1024 ** 3
}

total_bytes = 0

for s in sizes:
    match = re.match(r'([\d.]+)\s*(B|kB|MB|GB)', s)
    if match:
        number, unit = match.groups()
        bytes_size = float(number) * unit_multipliers[unit]
        total_bytes += bytes_size
    else:
        print(f"Unrecognized size format: {s}")

# Optional: convert back to human-readable
def human_readable(size_bytes):
    for unit in ['B', 'kB', 'MB', 'GB']:
        if size_bytes < 1024 or unit == 'GB':
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024

print(f"Total: {human_readable(total_bytes)}")
print("NOTE: This is the size of dbc file (compressed).")

Total: 4.22 MB
NOTE: This is the size of dbc file (compressed).


## Extract
### Downlaod all APAC Quimioterapy from 2022 of all BR States

In [11]:
# # Download all files of 2022
# sia.download(files, local_dir='../data/raw/')

Check if all files were downloaded

In [12]:
download_dir = Path('../data/raw')
all_downloaded_files = [f.name for f in download_dir.iterdir() if f.suffix == '.parquet']
print(f'All downloaded files: {len(all_downloaded_files)} of {all_apac_quimio_sia} of SIA')

All downloaded files: 321 of 321 of SIA


## Transform
### Convert to dataframe and merge all files into one

In [13]:
all_data_df_list = [ x.to_dataframe() for x in sia.download(files, local_dir='../data/raw/') ] 

132473it [00:00, 1589800.32it/s]       


In [14]:
print("Check type:")
print([type(x) for x in all_data_df_list[:3]])
print('\n')
print("Check dataframe n:")
print(len(all_data_df_list))


Check type:
[<class 'pandas.core.frame.DataFrame'>, <class 'pandas.core.frame.DataFrame'>, <class 'pandas.core.frame.DataFrame'>]


Check dataframe n:
321


In [15]:
# Concatenate to a single dataframe
combined_df = pd.concat(all_data_df_list, ignore_index=True)

print(f"Rows (Atendimentos) {combined_df.shape[0]:,.0f} \nColumns (Campos) {combined_df.shape[1]:,.0f} ")

Rows (Atendimentos) 3,985,483 
Columns (Campos) 74 


In [16]:
print(combined_df.columns.tolist())

['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'AQ_CID10', 'AQ_LINFIN', 'AQ_ESTADI', 'AQ_GRAHIS', 'AQ_DTIDEN', 'AQ_TRANTE', 'AQ_CIDINI1', 'AQ_DTINI1', 'AQ_CIDINI2', 'AQ_DTINI2', 'AQ_CIDINI3', 'AQ_DTINI3', 'AQ_CONTTR', 'AQ_DTINTR', 'AQ_ESQU_P1', 'AQ_TOTMPL', 'AQ_TOTMAU', 'AQ_ESQU_P2', 'AQ_MED01', 'AQ_MED02', 'AQ_MED03', 'AQ_MED04', 'AQ_MED05', 'AQ_MED06', 'AQ_MED07', 'AQ_MED08', 'AQ_MED09', 'AQ_MED10', 'AP_NATJUR']


In [17]:
combined_df.head(10)

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,AQ_MED02,AQ_MED03,AQ_MED04,AQ_MED05,AQ_MED06,AQ_MED07,AQ_MED08,AQ_MED09,AQ_MED10,AP_NATJUR
0,202201,EP,120000,2001586,1222200016158,202201,0304040045,1300.00,120040,05,...,,,,,,,,,,1147
1,202201,EP,120000,2001586,1221200274998,202201,0304020176,571.50,120040,05,...,,,,,,,,,,1147
2,202201,EP,120000,2001586,1221200244814,202201,0304050113,79.75,120040,05,...,,,,,,,,,,1147
3,202201,EP,120000,2001586,1222200035386,202201,0304020206,800.00,120040,05,...,,,,,,,,,,1147
4,202201,EP,120000,2001586,1221200276516,202201,0304050121,79.75,120040,05,...,,,,,,,,,,1147
5,202201,EP,120000,2001586,1221200243186,202201,0304020133,1700.00,120040,05,...,,,,,,,,,,1147
6,202201,EP,120000,2001586,1221200275460,202201,0304020419,1700.00,120040,05,...,,,,,,,,,,1147
7,202201,EP,120000,2001586,1222200016576,202201,0304020389,571.50,120040,05,...,,,,,,,,,,1147
8,202201,EP,120000,2001586,1221200244946,202201,0304040029,1400.00,120040,05,...,,,,,,,,,,1147
9,202201,EP,120000,2001586,1222200035199,202201,0304050040,79.75,120040,05,...,,,,,,,,,,1147


### Better naming


In [18]:
new_names_df = pd.read_csv('../data/external/better_names_apac_quimio.csv')

my_dict = new_names_df.set_index('original_name')['new_name'].to_dict()
my_dict

{'AP_MVM': 'data_movimento',
 'AP_CONDIC': 'tipo_gestao',
 'AP_GESTAO': 'codigo_gestao',
 'AP_CODUNI': 'estabelecimento_id',
 'AP_AUTORIZ': 'numero_apac',
 'AP_CMP': 'data_competencia',
 'AP_PRIPAL': 'procedimento_principal',
 'AP_VL_AP': 'valor_aprovado_total',
 'AP_UFMUN': 'municipio_estabelecimento',
 'AP_TPUPS': 'tipo_estabelecimento',
 'AP_TIPPRE': 'tipo_prestador',
 'AP_MN_IND': 'modalidade_estabelecimento',
 'AP_CNPJCPF': 'cnpj_estabelecimento',
 'AP_CNPJMNT': 'cnpj_mantenedora',
 'AP_CNSPCN': 'cns_paciente',
 'AP_COIDADE': 'codigo_idade',
 'AP_NUIDADE': 'idade',
 'AP_SEXO': 'sexo',
 'AP_RACACOR': 'raca_cor',
 'AP_MUNPCN': 'municipio_residencia',
 'AP_UFNACIO': 'nacionalidade',
 'AP_CEPPCN': 'cep_paciente',
 'AP_UFDIF': 'uf_diferente',
 'AP_MNDIF': 'municipio_diferente',
 'AP_DTINIC': 'data_inicio_validade',
 'AP_DTFIM': 'data_fim_validade',
 'AP_TPATEN': 'tipo_atendimento',
 'AP_TPAPAC': 'tipo_apac',
 'AP_MOTSAI': 'motivo_saida',
 'AP_OBITO': 'indicador_obito',
 'AP_ENCERR': 'i

In [19]:
combined_df = combined_df.rename(columns=my_dict)
combined_df

,data_movimento,tipo_gestao,codigo_gestao,estabelecimento_id,numero_apac,data_competencia,procedimento_principal,valor_aprovado_total,municipio_estabelecimento,tipo_estabelecimento,...,aq_med02,aq_med03,aq_med04,aq_med05,aq_med06,aq_med07,aq_med08,aq_med09,aq_med10,natureza_juridica
0,202201,EP,120000,2001586,1222200016158,202201,0304040045,1300.00,120040,05,...,,,,,,,,,,1147
1,202201,EP,120000,2001586,1221200274998,202201,0304020176,571.50,120040,05,...,,,,,,,,,,1147
2,202201,EP,120000,2001586,1221200244814,202201,0304050113,79.75,120040,05,...,,,,,,,,,,1147
3,202201,EP,120000,2001586,1222200035386,202201,0304020206,800.00,120040,05,...,,,,,,,,,,1147
4,202201,EP,120000,2001586,1221200276516,202201,0304050121,79.75,120040,05,...,,,,,,,,,,1147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985478,202212,EP,170000,2786117,1722200418598,202212,0304050121,79.75,172100,05,...,,,,,,,,,,1023
3985479,202212,EP,170000,2786117,1722200449992,202212,0304050121,79.75,172100,05,...,,,,,,,,,,1023
3985480,202212,EP,170000,2786117,1722200399106,202212,0304060178,1743.12,172100,05,...,,,,,,,,,,1023
3985481,202212,EP,170000,2600536,1722200409820,202212,0304020435,1700.00,170210,05,...,,,,,,,,,,1023


Export processed table

### Add CID

In [20]:
cid10_data = pd.read_csv('../data/external/CID-10-SUBCATEGORIAS.CSV.utf8',sep = ';')
cid10_data = cid10_data[['SUBCAT','DESCRICAO']]

cid10_data.rename(columns={"SUBCAT": 'CID10', "DESCRICAO":"cid10_descricao"}, inplace=True)
cid10_data

,CID10,cid10_descricao
0,A000,"Cólera devida a Vibrio cholerae 01, biótipo ch..."
1,A001,"Cólera devida a Vibrio cholerae 01, biótipo El..."
2,A009,Cólera não especificada
3,A010,Febre tifóide
4,A011,Febre paratifóide A
...,...,...
12446,U818,Agente resistente a outros antibióticos relaci...
12447,U88,Agente resistente a múltiplos antibióticos
12448,U898,Agente resistente a outro antibiótico especifi...
12449,U899,Agente resistente a antibiótico não especificado


In [21]:
combined_df = pd.merge(combined_df, cid10_data, how = 'left', left_on = 'cid_principal', right_on = 'CID10')

In [22]:
summary_cid = combined_df.groupby('cid10_descricao').size().reset_index(name='n').sort_values('n',  ascending=False)
summary_cid.head(50).to_csv('../data/interim/top50cids.csv', index = False)

In [23]:
cid_simplificado = pd.read_csv('../data/external/CID10_Simplificado.csv')
cid_simplificado = cid_simplificado[['cid10_descricao','cid10_descricao_simplificada']]

combined_df = pd.merge(combined_df, cid_simplificado, how = 'left', on = 'cid10_descricao')

In [24]:
# if cid10_descricao_simplificada is na, move to other
combined_df['cid10_descricao_simplificada'] = combined_df['cid10_descricao_simplificada'].fillna('Other')
combined_df.head(10)

,data_movimento,tipo_gestao,codigo_gestao,estabelecimento_id,numero_apac,data_competencia,procedimento_principal,valor_aprovado_total,municipio_estabelecimento,tipo_estabelecimento,...,aq_med05,aq_med06,aq_med07,aq_med08,aq_med09,aq_med10,natureza_juridica,CID10,cid10_descricao,cid10_descricao_simplificada
0,202201,EP,120000,2001586,1222200016158,202201,0304040045,1300.00,120040,05,...,,,,,,,1147,C539,"Neoplasia maligna do colo do útero, não especi...",Câncer de Colo do Útero
1,202201,EP,120000,2001586,1221200274998,202201,0304020176,571.50,120040,05,...,,,,,,,1147,C151,Neoplasia maligna da porção torácica do esôfag...,Other
2,202201,EP,120000,2001586,1221200244814,202201,0304050113,79.75,120040,05,...,,,,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama
3,202201,EP,120000,2001586,1222200035386,202201,0304020206,800.00,120040,05,...,,,,,,,1147,C029,"Neoplasia maligna da língua, não especificada",Other
4,202201,EP,120000,2001586,1221200276516,202201,0304050121,79.75,120040,05,...,,,,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama
5,202201,EP,120000,2001586,1221200243186,202201,0304020133,1700.00,120040,05,...,,,,,,,1147,C509,"Neoplasia maligna da mama, não especificada",Câncer de Mama
6,202201,EP,120000,2001586,1221200275460,202201,0304020419,1700.00,120040,05,...,,,,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama
7,202201,EP,120000,2001586,1222200016576,202201,0304020389,571.50,120040,05,...,,,,,,,1147,C229,"Neoplasia maligna do fígado, não especificada",Other
8,202201,EP,120000,2001586,1221200244946,202201,0304040029,1400.00,120040,05,...,,,,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama
9,202201,EP,120000,2001586,1222200035199,202201,0304050040,79.75,120040,05,...,,,,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama


### Add Estados

In [25]:
codigo_estados = pd.read_csv("../data/external/estados.csv")
codigo_estados.rename({'COD':'codigo_estado', 'NOME':'nome_estado', 'SIGLA':'sigla_estado'}, axis = 1, inplace=True)

codigo_estados


,codigo_estado,nome_estado,sigla_estado
0,35,São Paulo,SP
1,41,Paraná,PR
2,42,Santa Catarina,SC
3,43,Rio Grande do Sul,RS
4,50,Mato Grosso do Sul,MS
5,11,Rondônia,RO
6,12,Acre,AC
7,13,Amazonas,AM
8,14,Roraima,RR
9,15,Pará,PA


In [26]:
combined_df['codigo_estado'] = combined_df.municipio_estabelecimento.str[:2].astype(int)

In [27]:
combined_df = pd.merge(combined_df, codigo_estados, how = 'left', on = 'codigo_estado')
combined_df

,data_movimento,tipo_gestao,codigo_gestao,estabelecimento_id,numero_apac,data_competencia,procedimento_principal,valor_aprovado_total,municipio_estabelecimento,tipo_estabelecimento,...,aq_med08,aq_med09,aq_med10,natureza_juridica,CID10,cid10_descricao,cid10_descricao_simplificada,codigo_estado,nome_estado,sigla_estado
0,202201,EP,120000,2001586,1222200016158,202201,0304040045,1300.00,120040,05,...,,,,1147,C539,"Neoplasia maligna do colo do útero, não especi...",Câncer de Colo do Útero,12,Acre,AC
1,202201,EP,120000,2001586,1221200274998,202201,0304020176,571.50,120040,05,...,,,,1147,C151,Neoplasia maligna da porção torácica do esôfag...,Other,12,Acre,AC
2,202201,EP,120000,2001586,1221200244814,202201,0304050113,79.75,120040,05,...,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama,12,Acre,AC
3,202201,EP,120000,2001586,1222200035386,202201,0304020206,800.00,120040,05,...,,,,1147,C029,"Neoplasia maligna da língua, não especificada",Other,12,Acre,AC
4,202201,EP,120000,2001586,1221200276516,202201,0304050121,79.75,120040,05,...,,,,1147,C506,Neoplasia maligna da porção axilar da mama,Câncer de Mama,12,Acre,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985478,202212,EP,170000,2786117,1722200418598,202212,0304050121,79.75,172100,05,...,,,,1023,C500,Neoplasia maligna do mamilo e aréola,Câncer de Mama,17,Tocantins,TO
3985479,202212,EP,170000,2786117,1722200449992,202212,0304050121,79.75,172100,05,...,,,,1023,C500,Neoplasia maligna do mamilo e aréola,Câncer de Mama,17,Tocantins,TO
3985480,202212,EP,170000,2786117,1722200399106,202212,0304060178,1743.12,172100,05,...,,,,1023,NaN,NaN,Other,17,Tocantins,TO
3985481,202212,EP,170000,2600536,1722200409820,202212,0304020435,1700.00,170210,05,...,,,,1023,C508,Neoplasia maligna da mama com lesão invasiva,Câncer de Mama,17,Tocantins,TO


### Load

Our load will be just export to `.csv`. Nice :)

In [28]:
combined_df.to_csv('../data/processed/apac-quimio-2022.csv')

In [32]:
file_path = Path('../data/processed/apac-quimio-2022.csv')
size_bytes = file_path.stat().st_size
print('../data/processed/apac-quimio-2022.csv')
print(f"Size in GB: {size_bytes / (1024 ** 3):.2f}")

../data/processed/apac-quimio-2022.csv
Size in GB: 1.65


In [34]:
toy_data = combined_df.sample(frac = 0.1, axis = 1)
toy_data.to_csv('../data/processed/apac-quimio-2022-toy.csv')

file_path = Path('../data/processed/apac-quimio-2022-toy.csv')
size_bytes = file_path.stat().st_size
print('../data/processed/apac-quimio-2022-toy.csv')
print(f"Size in GB: {size_bytes / (1024 ** 3):.2f}")

../data/processed/apac-quimio-2022-toy.csv
Size in GB: 0.15
